In [1]:
from pycocotools.coco import COCO
import os
import os.path as osp
import sys
import torch
import torch.utils.data as data
import torchvision.transforms as transforms
import cv2
import numpy as np
from pycocotools.coco import COCO
from torch.utils.data import DataLoader
import numpy as np
from PIL import Image

val_info = r"D:\Praktikum\datasets\coco\annotations_trainval2017\annotations\instances_val2017.json"
val_image = r"D:\Praktikum\datasets\coco\val2017\val2017"
val_cap_info = r"D:\Praktikum\datasets\coco\annotations_trainval2017\annotations\captions_val2017.json"

class COCODetection(data.Dataset):
    def __init__(self, image_path, info_file, has_gt=True):
        self.root = image_path
        self.coco = COCO(info_file)
        self.ids = list(self.coco.imgToAnns.keys())  
        self.coco_caps=COCO(val_cap_info)

        if len(self.ids) == 0 or not has_gt:  
            self.ids = list(self.coco.imgs.keys())
        
        

        self.has_gt = has_gt

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, index):
        im ,gt, captions,file_name, h, w, num_crowds = self.pull_item(index)
        
        return im,captions,file_name

    def pull_item(self, index):
        img_id = self.ids[index]
        
        if self.has_gt:
            ann_ids = self.coco.getAnnIds(imgIds=img_id)
            target = self.coco.loadAnns(ann_ids)
            
            
        else:
            target = []
        
        ann_cap_ids = self.coco_caps.getAnnIds(imgIds=img_id)
        ann_cap = self.coco_caps.loadAnns(ann_cap_ids)
        
        crowd = [x for x in target if ('iscrowd' in x and x['iscrowd'])]
        target = [x for x in target if not ('iscrowd' in x and x['iscrowd'])]
        num_crowds = len(crowd)

        
        target += crowd
        
        file_name = self.coco.loadImgs(img_id)[0]['file_name']
        path = osp.join(self.root, file_name)
        img = cv2.imread(path)
        
        
        captions = []
        for ann in ann_cap:
            captions.append(ann['caption'])
    

        
        

        
        img = cv2.resize(img, (256,256))
        
        height, width, _ = img.shape

        
        
        return torch.from_numpy(img), target, captions,file_name,height, width, num_crowds


if __name__=='__main__':
    dataset = COCODetection(val_image, val_info)
    loader = DataLoader(dataset,batch_size=1)
    for n, (img,captions,file_name) in enumerate(loader):

        
        img = np.uint8(img.squeeze().numpy())
        #cv2.imshow('img', img)
        print(file_name)
        print(captions)
        
        
        #gt, masks, num_crowds = label
        #masks = masks.squeeze(0)
        """
        for m in range(masks.size(0)):
            mask = masks[m].numpy()
            color = np.random.randint(0, 255)
            channel = np.random.randint(0, 3)
            y, x = np.where(mask == 1)
            img[y, x, channel] = color
        """
        cv2.imshow('img', img)
        
        cv2.waitKey(500)

d:\Anaconda\envs\VQ-Diffusion\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Anaconda\envs\VQ-Diffusion\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


loading annotations into memory...
Done (t=0.48s)
creating index...
index created!
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
('000000289343.jpg',)
[('A man is riding a large bike through the park.',), ('A person riding a very tall bike in the street.',), ('A man this is on a high wheel bicycle.',), ('A person riding a bicycle on a deserted street.',), ('A person rides a bike on the road.',)]
('000000061471.jpg',)
[('Puppy chewing on toilet paper in the bathroom. ',), ('A brown dog standing next to a toilet in a bathroom.',), ('A puppy pulls a line of toilet paper. ',), ('A dog playing with a roll of toilet paper in a bathroom.',), ('A puppy has pulled toilet paper across the bathroom floor.',)]
('000000472375.jpg',)
[('A small black and brown dog sitting next to a motorcycle.',), ('a brown black and white dog wearing a helmet and a black scooter',), ('A German Shepherd dog wearing a helmet sitting by a motor cycle.',), ('A dog is wearing a helme

KeyboardInterrupt: 

: 